In [25]:
import dataclasses
import numpy as np
import pandas as pd
import toml
import pathlib2 as pathlib


with open("config.toml") as f:
    config = toml.load(f)

@dataclasses.dataclass
class Thresholds:
    GQ: int = config['QC_Thresholds']['GQ']
    DP: int = config['QC_Thresholds']['DP']
    AB_het = config['QC_Thresholds']['AB_het']
    AB_hom = config['QC_Thresholds']['AB_hom']
    SIFT: float = config['In_silico_predictions']['SIFT']
    PP2: float = config['In_silico_predictions']['PolyPhen-2']
    CADD: float = config['In_silico_predictions']['CADD']
    MutationTaster: list = dataclasses.field(default_factory=list)
    GGM_AF: float = config['AF_Thresholds']['GGM']


# Define the thresholds of 'MutationTaster' as a list of strings
Thresholds.MutationTaster = config['In_silico_predictions']['exclude_MutationTaster']

In [26]:
input_dir = '/Volumes/vol/work/Github/TestData/ggm'
input_csv = f"{input_dir}/ggm_trio_SEL1L.csv"
input_csv = pathlib.Path(input_csv)
input_basename = input_csv.stem

In [29]:
def _split_qc_col(df: pd.DataFrame) -> pd.DataFrame:
    for rel in ['pro', 'pat', 'mat']:
        df = pd.concat(
            [df, df[f'GQ:DP:AD({rel})'].str.split(':', expand=True)], axis=1)
        for i in range(3):
            df[i] = df[i].replace('.', np.nan)
            df[i] = df[i].replace('-', np.nan)
        df = df.astype({0: float, 1: float, 2: float})
        df.rename(columns={0: f'GQ({rel})', 1: f'DP({rel})', 2: f'AD({rel})'}, 
                inplace=True)
    return df

def _add_ab_col(df: pd.DataFrame) -> pd.DataFrame:
    for rel in ['pro', 'pat', 'mat']:
        df[f'AB({rel})'] = df[f'AD({rel})'] / df[f'DP({rel})']
    return df

def _split_ggmacan_col(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.concat(
        [df, df['GGM(AC/AN)'].str.split('/', expand=True)], axis=1)
    df = df.astype({0: float, 1: float})
    df.rename(columns={0: 'GGM(AC)', 1: 'GGM(AN)'}, inplace=True)

    return df


def _add_ggmaf_col(df: pd.DataFrame) -> pd.DataFrame:
    df['GGM(AF)'] = df['GGM(AC)'] / df['GGM(AN)']
    
    return df

def _rename_cols(df: pd.DataFrame) -> pd.DataFrame:
    rename_dict: dict = {
        'Chr': 'CHROM',
        'Position': 'POS',
        'Ref': 'REF',
        'Alt': 'ALT'
        }
    df.rename(columns=rename_dict, inplace=True)

    return df

def generate_variant_id(df: pd.DataFrame) -> pd.DataFrame:
    df['variant_id'] = df['Chr'] + ':' + \
                       df['Position'] + '-' + df['Ref'] + '-' + df['Alt']
    return df

def add_qc_filter(row, thresholds: Thresholds) -> pd.DataFrame:    
    if row['GQ(pro)'] < thresholds.GQ:
        return '.'
    else:
        if row['Vtype'] == 'homo':
            if row['AB(pro)'] >= (1 - thresholds.AB_hom):
                return 'PASS'
            else:
                return '.'
        else:
            if thresholds.AB_het <= row['AB(pro)'] <= (1 - thresholds.AB_het):
                return 'PASS'
            else:
                return '.'

def add_insilico_filter(row, thresholds: Thresholds) -> str:
    if ((row['SIFT'] >= thresholds.SIFT) 
        and (row['PolyPhen-2'] <= thresholds.PP2)
        and (row['CADD'] < thresholds.CADD)
        and (row['MutationTaster'] in thresholds.MutationTaster)):
        return 'FAIL' 
    else:
        return 'PASS'

def add_identified_filter(df: pd.DataFrame) -> pd.DataFrame:
    df.loc[
        df['Analysis status'] != 'Identified', 
        'Not_Identified_FILTER'] = 'PASS'

    return df

def add_ggmmaf_filter(df: pd.DataFrame, thresholds: Thresholds) -> pd.DataFrame:
    df.loc[
        df['GGM(AF)'] < thresholds.GGM_AF,
        'GGM_FILTER'] = 'PASS'
    
    return df
    
def add_hard_filter(df: pd.DataFrame) -> pd.DataFrame:
    df.loc[
        (
            (df['QC_FILTER'] == 'PASS')
            & (df['Not_Identified_FILTER'] == 'PASS')
            & (df['insilico_FILTER'] == 'PASS')
            & (df['GGM_FILTER'] == 'PASS')
        ),
        'HARD_FILTER'] = 'PASS'
    
    return df

# Pre-processing functions
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = _split_qc_col(df)
    df = _add_ab_col(df)
    df = _split_ggmacan_col(df)
    df = _add_ggmaf_col(df)
    df = generate_variant_id(df)
    df.replace({'SIFT': '-', 'PolyPhen-2': '-', 'CADD': '-'}, np.nan, inplace=True)
    df = df.astype(
        {'SIFT': float, 'PolyPhen-2': float, 'CADD': float, 
         'GQ(pro)': float, 'AB(pro)': float, 'GGM(AF)': float}
        )

    return df

def reorder_columns(df: pd.DataFrame) -> pd.DataFrame:
    # Drop the 1st column
    df = df.drop(columns=df.columns[0])
    
    # Rename the columns
    df = _rename_cols(df)

    # Reorder the columns
    reodered_columns = [
        'Gene', 'Transcript', 'Family', 'Sample', 'Disease', 
        'Vtype', 'variant_id', 'Amino acid change2', 'Effect',  
        'Distance',  'SIFT', 'PolyPhen-2', 'MutationTaster', 'CADD', 'HARD_FILTER', 
        'gnomAD(AF)', 'ExAC(AF)', 'ToMMo3.5K(AF)', 'GGM(AF)', 'JPNCTL(SC)', 
        'GGM(AC)', 'gnomAD(AC)', 'ToMMo3.5K(AC)',  
        'ID(pro)', 'AS(pro)', 'GT(pro)', 'GQ(pro)', 'DP(pro)', 'AD(pro)', 'AB(pro)',
        'ID(pat)', 'AS(pat)', 'GT(pat)', 'GQ(pat)', 'DP(pat)', 'AD(pat)', 'AB(pat)',
        'ID(mat)', 'AS(mat)', 'GT(mat)', 'GQ(mat)', 'DP(mat)', 'AD(mat)', 'AB(mat)',
        'Impact', 'QC_FILTER', 'Not_Identified_FILTER', 'insilico_FILTER',
        'Analysis status', 'Identified gene', 'Variant description', 
        'CHROM', 'POS', 'REF', 'ALT'
        ]
    
    df = df[reodered_columns]

    return df

def postprocess(df: pd.DataFrame) -> pd.DataFrame:
    df.replace(np.nan, '.', inplace=True)

    return df

In [30]:
df = pd.read_csv(input_csv, header=0, dtype=str)

df = preprocess(df)
df = add_identified_filter(df)
df = add_ggmmaf_filter(df, Thresholds())
df['QC_FILTER'] = df.apply(add_qc_filter, args=(Thresholds(),), axis=1)
df['insilico_FILTER'] = df.apply(add_insilico_filter, args=(Thresholds(),), axis=1)
df = add_hard_filter(df)
df = reorder_columns(df)
df = postprocess(df)


In [31]:
df

,Gene,Transcript,Family,Sample,Disease,Impact,Vtype,variant_id,Amino acid change2,Effect,...,QC_FILTER,Not_Identified_FILTER,insilico_FILTER,Analysis status,Identified gene,Variant description,CHROM,POS,REF,ALT
0,SEL1L,NM_005065.5,28443,Sample_28443,Kyphomelic_dysplasia,MODIFIER,denovo,14:81953889-GA-G,c.1484-8delT,intron_variant,...,.,PASS,PASS,-,-,-,14,81953889,GA,G
1,SEL1L,NM_005065.5,33398,Sample_33398,Cerebellar_Ataxia,MODIFIER,denovo,14:81953889-G-GAAA,c.1484-7_1484-8insTTT,intron_variant,...,.,PASS,PASS,-,-,-,14,81953889,G,GAAA
2,SEL1L,NM_005065.5,33703,Sample_33703,Dystonia,MODIFIER,denovo,14:81953889-G-GA,c.1484-7_1484-8insT,intron_variant,...,.,PASS,PASS,-,-,-,14,81953889,G,GA
3,SEL1L,NM_005065.5,28529,Sample_28529,Hemidystonia_Regression,MODIFIER,denovo,14:81953889-GAA-G,c.1484-9_1484-8delTT,intron_variant,...,.,PASS,PASS,Undetermined,-,-,14,81953889,GAA,G
4,SEL1L,NM_005065.5,29213,Sample_29213,Leigh_syndrome,MODIFIER,denovo,14:81953889-GA-G,c.1484-8delT,intron_variant,...,.,nan,PASS,Identified,MT_ND3,m.10191T>C:p.(S45P),14,81953889,GA,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,SEL1L,NM_005065.5,13572,Sample_13572,Congenital_scoliosis,LOW,ch_mo,14:81970643-T-C,c.516A>G:p.Glu172Glu,synonymous_variant,...,PASS,PASS,PASS,-,-,-,14,81970643,T,C
245,SEL1L,NM_005065.5,13572,Sample_13572,Congenital_scoliosis,MODIFIER,ch_fa,14:82000003-G-T,c.70+16C>A,intron_variant,...,PASS,PASS,PASS,-,-,-,14,82000003,G,T
246,SEL1L,NM_005065.5,16736,Sample_16736,Pachygyria:Agenesis_of_corpus_callosum,MODIFIER,ch_fa,14:81972400-C-T,c.508+18G>A,intron_variant,...,PASS,PASS,PASS,Undetermined,-,-,14,81972400,C,T
247,SEL1L,NM_005065.5,29339,Sample_29339,hyaloideoretinal_degeneration_heart_anormaly,MODIFIER,denovo,14:81953889-GA-G,c.1484-8delT,intron_variant,...,PASS,PASS,PASS,Undetermined,-,-,14,81953889,GA,G


In [ ]:
df.to_excel(f"{input_dir}/{input_basename}.parsed.xlsx", index=False)

In [18]:
df['Vtype'].count_values()

AttributeError: 'Series' object has no attribute 'count_values'